## CNN 

In [1]:
from PyFunctions import Functions as func
from PyFunctions import var
from PyFunctions import ModelFunc as modelfunc
import os
from tqdm import tqdm
import numpy as np
import cv2

Using plaidml.keras.backend backend.


### Video 
- within the folder Tests, add whatever video (.mp4) that contains a gun and run the cell below
- **Note**: Stick t video shorter than a minute or this process will take a very long time depending on your computer

In [2]:
def get_vid_frames(path, model, dim, edge = False): 
    '''This function will take a path to an mp4 file and return a list containing each frame of the video.  This function is used for creating bounding boxes within a video'''
    from tqdm import tqdm
    vid = cv2.VideoCapture(path)
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total = total_frames, desc = f'Splitting Video Into {total_frames} Frames')
    images = [] 
    sucess =1 
    while True: 
        try:
            success, img = vid.read() 
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            images.append(img)
            pbar.update(1)
        except: 
            break
        

    pbar.close()
    images = np.array(images)
    
    clones = []
    for img in tqdm(images, desc = 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While'): 
        if edge == True:
            img2 = img.reshape(1, img.shape[0], img.shape[1], 1)
        else: 
            img2 = img.reshape(1, img.shape[0], img.shape[1], 3)
        clone = img.copy()

        pred = model.predict(img2)[0]

        category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
        cat_index = np.argmax(pred)
        cat = category_dict[cat_index]
        #if the prediction is non_weapon then continue to the next without creating a bounding box
        if cat_index == 0: 
            clones.append(clone)
            continue
            
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        ss.setBaseImage(img)
    #     ss.switchToSelectiveSearchQuality()
        ss.switchToSelectiveSearchFast()

        rects = ss.process() 

        windows = []
        locations = []
        for x, y, w,h in rects[:1001]: 
            startx = x 
            starty = y 
            endx = x+w 
            endy = y+h 
            roi = img[starty:endy, startx:endx]
            if edge == True:
                roi = get_edged(roi, dim = dim)
            roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
            windows.append(roi)
            locations.append((startx, starty, endx, endy))

        windows = np.array(windows)
        if edge == True:
            windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
        else: 
            windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
        windows = np.array(windows)
        locations = np.array(locations)

        predictions = model.predict(windows)
        nms = func.non_max_suppression(locations)
        bounding_cnt = 0
        for idx in nms:
            if np.argmax(predictions[idx]) != cat_index: 
                continue
            startx, starty, endx, endy = locations[idx]
            cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
            text = f'{category_dict[np.argmax(predictions[idx])]}: {int(predictions[idx].max()*100)}%'
            cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
            bounding_cnt += 1

        if bounding_cnt == 0: 
            pred_idx= [idx for idx, i in enumerate(predictions) if np.argmax(i) == cat_index]
            cat_locations = np.array([locations[i] for i in pred_idx])
            nms = func.non_max_suppression(cat_locations)
            if len(nms)==0:
                cat_predictions = predictions[:,cat_index]
                pred_max_idx = np.argmax(cat_predictions)
                pred_max = cat_predictions[pred_max_idx]

                pred_max_window = locations[pred_max_idx]
                startx, starty, endx, endy = pred_max_window
                cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
                text = f'{category_dict[cat_index]}: {int(pred_max*100)}%'
                cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
            for idx in nms: 
                startx, starty, endx, endy = cat_locations[idx]
                cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
                text = f'{category_dict[np.argmax(predictions[pred_idx[idx]])]}: {int(predictions[pred_idx[idx]].max()*100)}%'
                cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
        clones.append(clone)
        
        vid_dim = dim
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('Tests/VideoOutput/Test.mp4',fourcc, 10, vid_dim) #change the filename to whatever you would like

        out_writer = [out.write(i) for i in clones]
        out.release()
        cv2.destroyAllWindows()
    return clones

In [ ]:
edge= False 

dim = (var.norm_dimension[0],var.norm_dimension[1], 3)
    
normal_model = modelfunc.get_conv_model(dim)
normal_model.load_weights('ModelWeights/Normal/V2_NoEdge_NoAugmentation.h5')

images = get_vid_frames('../Tests/Videos/Pistol2.mp4', normal_model, var.norm_dimension, edge = edge)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"
Splitting Video Into 720 Frames: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 720/720 [00:02<00:00, 308.54it/s]
Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While:   4%|██▎                                                      | 29/720 [00:32<13:43,  1.19s/it]

In [ ]:
assert False

In [ ]:
def get_img_prediction_bounding_box(path, model, dim, edge = False, model_type = 'normal'):
    '''This function will create a bounding box over what it believes is a weapon given the image path, dimensions, and model used to detect the weapon.  Dimensions can be found within the Var.py file.  This function is still being used as I need to apply non-max suppresion to create only one bounding box'''
    img = get_image_value(path, dim, edge = edge)

    if edge == True:
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else: 
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    
    pred = model.predict(img)[0]
    
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    print(f'{path}\t\tPrediction: {cat}\t{int(pred.max()*100)}% Confident')
    
    
    #speed up cv2
    cv2.setUseOptimized(True)
    cv2.setNumThreads(10)
    
    img = cv2.imread(path)

    clone = img.copy() 
    clone2 = img.copy()
    
#     if cat_index != 0:
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
#     ss.switchToSelectiveSearchQuality()
    ss.switchToSelectiveSearchFast()

    rects = ss.process() 

    windows = []
    locations = []
    print(f'Creating Bounding Boxes for {path}')
    for x, y, w,h in rects[:1001]: 
        startx = x 
        starty = y 
        endx = x+w 
        endy = y+h 
        roi = img[starty:endy, startx:endx]
        if edge == True:
            roi = get_edged(roi, dim = dim)
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))

    windows = np.array(windows)
    if edge == True:
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
    else: 
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    windows = np.array(windows)
    locations = np.array(locations)
    
#     if model_type == 'mobilenet': 
#         from keras.applications.mobilenet import preprocess_input
#         windows = preprocess_input(windows)
    predictions = model.predict(windows)
    nms = non_max_suppression(locations)
    bounding_cnt = 0
    for idx in nms:
        if np.argmax(predictions[idx]) != cat_index: 
            continue
#         print(np.argmax(predictions[idx]), predictions[idx].max())
        startx, starty, endx, endy = locations[idx]
        cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        text = f'{category_dict[np.argmax(predictions[idx])]}: {int(predictions[idx].max()*100)}%'
        cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
        bounding_cnt += 1

    if bounding_cnt == 0: 
        pred_idx= [idx for idx, i in enumerate(predictions) if np.argmax(i) == cat_index]
        cat_locations = np.array([locations[i] for i in pred_idx])
        nms = non_max_suppression(cat_locations)
        if len(nms)==0:
            cat_predictions = predictions[:,cat_index]
            pred_max_idx = np.argmax(cat_predictions)
            pred_max = cat_predictions[pred_max_idx]

            pred_max_window = locations[pred_max_idx]
            startx, starty, endx, endy = pred_max_window
            cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
            text = f'{category_dict[cat_index]}: {int(pred_max*100)}%'
            cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
        for idx in nms: 
            startx, starty, endx, endy = cat_locations[idx]
            cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
            text = f'{category_dict[np.argmax(predictions[pred_idx[idx]])]}: {int(predictions[pred_idx[idx]].max()*100)}%'
            cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
               


    
#     pick = func.non_max_suppression(locations, probs = None)

#     for idx in pick: 
#         startx, startx, endx, endy = locations[idx]
#         cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    cv2.imshow(f'Test', np.hstack([clone, clone2]))
    cv2.waitKey(0)
    ss.clear()


    return predictions

In [ ]:
pbar = tqdm(images, desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = [window_prob_func(img, normal_model, var.norm_dimension, edge = edge) for img in pbar]

In [ ]:
vid_dim = (224,224)
vid_dim = var.norm_dimension
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Tests/VideoOutput/BoundingBox.mp4',fourcc, 10, vid_dim) #change the filename to whatever you would like


for prob, img in zip(windows_prob, images): 
    clone = img.copy()
    if prob == None:
        out.write(clone)
    else:
        (p, (startx, starty, endx, endy)) = prob
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
        out.write(clone)
        
out.release()
cv2.destroyAllWindows()